## Importacion de librerias a utilizar

In [1]:
import os
import pickle
import pandas as pd
# Gmail API utils
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode, urlsafe_b64encode
# for dealing with attachement MIME types
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from mimetypes import guess_type as guess_mime_type

# Request all access (permission to read/send/receive emails, manage the inbox, and more)
SCOPES = ['https://mail.google.com/']
our_email = 'luisfuturo@gmail.com'

## Definimos la funcion para cargar las credenciales 

In [2]:
def gmail_authenticate():
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

# get the Gmail API service
service = gmail_authenticate()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=586765979968-pf9as5c3k3fhaf64po8f9v7u50epek67.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55058%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=7oMVeX94Qp3KU1zB27LwVGT98TcWcp&access_type=offline


## Definimos la funcion para adjuntar arhivos al email

In [3]:
# Adds the attachment with the given filename to the given message
def add_attachment(message, filename):
    content_type, encoding = guess_mime_type(filename)
    if content_type is None or encoding is not None:
        content_type = 'application/octet-stream'
    main_type, sub_type = content_type.split('/', 1)
    if main_type == 'text':
        fp = open(filename, 'rb')
        msg = MIMEText(fp.read().decode(), _subtype=sub_type)
        fp.close()
    elif main_type == 'image':
        fp = open(filename, 'rb')
        msg = MIMEImage(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'audio':
        fp = open(filename, 'rb')
        msg = MIMEAudio(fp.read(), _subtype=sub_type)
        fp.close()
    else:
        fp = open(filename, 'rb')
        msg = MIMEBase(main_type, sub_type)
        msg.set_payload(fp.read())
        fp.close()
    filename = os.path.basename(filename)
    msg.add_header('Content-Disposition', 'attachment', filename=filename)
    message.attach(msg)

## Definimos la funcion que toma los parametros del mensaje y retorna el mensaje del email

In [4]:
def build_message(destination, obj, body, attachments=[]):
    if not attachments: # no attachments given
        message = MIMEText(body)
        message['to'] = destination
        message['from'] = our_email
        message['subject'] = obj
    else:
        message = MIMEMultipart()
        message['to'] = destination
        message['from'] = our_email
        message['subject'] = obj
        message.attach(MIMEText(body))
        for filename in attachments:
            add_attachment(message, filename)
    return {'raw': urlsafe_b64encode(message.as_bytes()).decode()}

## Funcion que toma los  parametros del mensaje y usa  Google mail API para enviar un mensaje construido con build_message()

In [5]:
def send_message(service, destination, obj, body, attachments=[]):
    return service.users().messages().send(
      userId="me",
      body=build_message(destination, obj, body, attachments)
    ).execute()

## Usemos la funcion que envia mensajes

In [6]:
send_message(service, "luisfuturo@domain.com", "This is me from the dark side", 
            "proceso de seleccion primeIT", ["local_source/Prueba_AnalisisDatos.csv"])

{'id': '183368f9c3274643',
 'threadId': '183368f9c3274643',
 'labelIds': ['SENT']}

## Esta funcion retornara las IDs de los emails que coincidiran con la query

In [12]:
def search_messages(service, query):
    result = service.users().messages().list(userId='me',q=query).execute()
    messages = [ ]
    if 'messages' in result:
        messages.extend(result['messages'])
    while 'nextPageToken' in result:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
        if 'messages' in result:
            messages.extend(result['messages'])
    return messages

## La funcion get_size_format()  solo retornara bytes, tendremos que utilizar la funcion clean()

In [7]:
# utility functions
def get_size_format(b, factor=1024, suffix="B"):
    """
    Scale bytes to its proper byte format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    for unit in ["", "K", "M", "G", "T", "P", "E", "Z"]:
        if b < factor:
            return f"{b:.2f}{unit}{suffix}"
        b /= factor
    return f"{b:.2f}Y{suffix}"


## Definimos la funcion clean 

In [8]:
def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

## Definimos la funcion que contiene la particion del email

In [9]:
def parse_parts(service, parts, folder_name, message):
    """
    Utility function that parses the content of an email partition
    """
    if parts:
        for part in parts:
            filename = part.get("filename")
            mimeType = part.get("mimeType")
            body = part.get("body")
            data = body.get("data")
            file_size = body.get("size")
            part_headers = part.get("headers")
            if part.get("parts"):
                # recursively call this function when we see that a part
                # has parts inside
                parse_parts(service, part.get("parts"), folder_name, message)
            if mimeType == "text/plain":
                # if the email part is text plain
                if data:
                    text = urlsafe_b64decode(data).decode()
                    print(text)
            elif mimeType == "text/html":
                # if the email part is an HTML content
                # save the HTML file and optionally open it in the browser
                if not filename:
                    filename = "index.html"
                filepath = os.path.join(folder_name, filename)
                print("Saving HTML to", filepath)
                with open(filepath, "wb") as f:
                    f.write(urlsafe_b64decode(data))
            else:
                # attachment other than a plain text or HTML
                for part_header in part_headers:
                    part_header_name = part_header.get("name")
                    part_header_value = part_header.get("value")
                    if part_header_name == "Content-Disposition":
                        if "attachment" in part_header_value:
                            # we get the attachment ID 
                            # and make another request to get the attachment itself
                            print("Saving the file:", filename, "size:", get_size_format(file_size))
                            attachment_id = body.get("attachmentId")
                            attachment = service.users().messages() \
                                        .attachments().get(id=attachment_id, userId='me', messageId=message['id']).execute()
                            data = attachment.get("data")
                            filepath = os.path.join(folder_name, filename)
                            if data:
                                with open(filepath, "wb") as f:
                                    f.write(urlsafe_b64decode(data))

## Definimos la funcion para leer emails

In [10]:
def read_message(service, message):
    """
    This function takes Gmail API `service` and the given `message_id` and does the following:
        - Downloads the content of the email
        - Prints email basic information (To, From, Subject & Date) and plain/text parts
        - Creates a folder for each email based on the subject
        - Downloads text/html content (if available) and saves it under the folder created as index.html
        - Downloads any file that is attached to the email and saves it in the folder created
    """
    msg = service.users().messages().get(userId='me', id=message['id'], format='full').execute()
    # parts can be the message body, or attachments
    payload = msg['payload']
    headers = payload.get("headers")
    parts = payload.get("parts")
    folder_name = "email"
    has_subject = False
    if headers:
        # this section prints email basic info & creates a folder for the email
        for header in headers:
            name = header.get("name")
            value = header.get("value")
            if name.lower() == 'from':
                # we print the From address
                print("From:", value)
            if name.lower() == "to":
                # we print the To address
                print("To:", value)
            if name.lower() == "subject":
                # make our boolean True, the email has "subject"
                has_subject = True
                # make a directory with the name of the subject
                folder_name = clean(value)
                # we will also handle emails with the same subject name
                folder_counter = 0
                while os.path.isdir(folder_name):
                    folder_counter += 1
                    # we have the same folder name, add a number next to it
                    if folder_name[-1].isdigit() and folder_name[-2] == "_":
                        folder_name = f"{folder_name[:-2]}_{folder_counter}"
                    elif folder_name[-2:].isdigit() and folder_name[-3] == "_":
                        folder_name = f"{folder_name[:-3]}_{folder_counter}"
                    else:
                        folder_name = f"{folder_name}_{folder_counter}"
                os.mkdir(folder_name)
                print("Subject:", value)
            if name.lower() == "date":
                # we print the date when the message was sent
                print("Date:", value)
    if not has_subject:
        # if the email does not have a subject, then make a folder with "email" name
        # since folders are created based on subjects
        if not os.path.isdir(folder_name):
            os.mkdir(folder_name)
    parse_parts(service, parts, folder_name, message)
    print("="*50)

## Obtenemos el email que coincidi con la query que hemos espeficicado

In [13]:
# get emails that match the query you specify
results = search_messages(service, "This is me from the dark side")
print(f"Found {len(results)} results.")
# for each email matched, read it (output plain/text to console & save HTML and attachments)
for msg in results:
    read_message(service, msg)

Found 9 results.
To: luisfuturo@domain.com
From: luisfuturo@gmail.com
Subject: This is me from the dark side
Date: Tue, 13 Sep 2022 04:15:24 -0700
proceso de seleccion primeIT
Saving the file: Prueba_AnalisisDatos.csv size: 4.46MB
From: Quora Digest <english-personalized-digest@quora.com>
To: luisfuturo@gmail.com
Subject: What if Russia invades Finland after things end with Ukraine?
Date: Tue, 10 May 2022 11:46:06 +0000
Top stories for Luis

-----

Question: What if Russia invades Finland after things end with Ukraine?


Answer from Jussi Salmi


Finland can send 900 000 men (including me) to the front. In a survey it was found that Finns are the most willing to fight for their country in Europe. All men and half o 
Read More: https://www.quora.com/qemail/tc?al_imp=eyJ0eXBlIjogMzMsICJoYXNoIjogIjE4MDk1NDAwMDI0MTA0NDU5MTR8MXwxfDM1MTA3NjYzNCJ9&al_pri=8&aoid=jqO9shsNo1H&aoty=1&aty=4&cp=1&et=2&id=8b8f74442b9e40abab28c602c7c4b275&uid=M3lfahSijc4
-----
Question: What is the dark side of Londo

## Cargamos el archivo csv que hemos enviado a nuestro email

In [15]:
df = pd.read_csv('local_destiny/Prueba_AnalisisDatos.csv')

In [16]:
df

,id_reserva,fecha_reserva,pvp_reserva,coste_reserva
0,72000000,1/8/2022,"799,57","794,68"
1,72000001,1/8/2022,832,"806,82"
2,72000002,1/8/2022,890,864
3,72000003,1/8/2022,"686,72","672,99"
4,72000004,1/8/2022,1053,1022
...,...,...,...,...
132764,72135785,31/8/2022,"1522,43","219,23"
132765,72135786,31/8/2022,"2327,78","344,05"
132766,72135787,31/8/2022,4597,"492,94"
132767,72135788,31/8/2022,"1297,42","173,78"


## Este codigo esta inspirado en el blog de Abdou Rockikz, https://www.thepythoncode.com/article/use-gmail-api-in-python